# Creador de DataSets

Automatizador para crear datasets falsos y poder facilmente usarlos en los ejercicios.

Utiliza [faker](https://github.com/joke2k/faker/). Instalarlo con `pip install faker`

Solamente reemplazar las constantes de la **segunda celda** (la primera es de setup) por lo pedido en el enunciado y después correr esa celda y la siguiente.

Las constantes de ejemplo son las usadas para el siguiente ejercicio:

>El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud).  Los  posibles estados  que puede tener  un caso son 1:
 caso abierto, 2: caso  resuelto, 3: cerrado sin resolución.  Las fechas se encuentran en el formato YYYY-MM-DD.[...]

In [8]:
import pandas as pd
import numpy as np
import datetime
import random
from faker import Faker

def lorem(cantidad_palabras=6,posibilidades=1000):
    strings = []
    fake = Faker()
    for i in range(posibilidades):
        s = ' '.join(fake.text().split()[:cantidad_palabras])
        s = s.replace('\n','')
        strings.append(s)
    return strings

def fechas(formato, desde, hasta, cantidad):
    fechas = []
    fake = Faker()
    desde = datetime.datetime.strptime(desde,'%d/%m/%Y')
    hasta = datetime.datetime.strptime(hasta,'%d/%m/%Y')
    for i in range(cantidad):
        fechas.append(fake.date_time_between(start_date=desde,end_date=hasta))   
    fechas = sorted(fechas)
    for i in range(len(fechas)):
        fechas[i] = datetime.datetime.strftime(fechas[i],formato)
    return fechas

In [9]:
# OUTPUT -> Nombre del archivo final
# CANTIDAD_REGISTROS -> Cantidad de filas que tiene el dataset
# COLUMNAS -> Copiar las del enunciado
# FORMATO:
#         Listas -> Hardcodear todos los valores posibles (por ejemplo, variables categoricas)
#                 Se puede usar list comprehension cuando tenemos minimos y maximos
#         lorem(cantidad_palabras, n) -> Devuelve n strings de x palabras al azar.
#         fechas(formato, desde, hasta, x) -> Llena de x fechas al azar ordenadas en el rango de los dias pasados por parametro, en el formato pedido.
#                 El formato pasado por parametro tiene que ser como el indicado en http://strftime.org/ (YYYY-MM-DD pasa a %Y-%m-%d)
#                 Las fechas pasadas por parametro deben estar en formato %d/%m/%Y
# RANDOM -> Indica con un booleano que valores deben ser randoms y cuales secuenciales. 
#                 Por ejemplo, las fechas estan ordenadas y ya de por si son azarosas, los ids estan ordenados secuencialmente, pero las variables categoricas deben ser al azar

OUTPUT = 'data/' + '2018C1_1_GCPD_Casos_Policiales' + '.csv'

CANTIDAD_REGISTROS = 20 # No usar un número demasiado grande, es mejor poder ver a ojo que los resultados sean correctos

COLUMNAS = ['fecha',
           'id_caso',
           'descripcion',
           'estado_caso',
           'categoria',
           'latitud',
           'longitud']

FORMATO = [fechas('%Y-%m-%d','1/1/2018','1/6/2018',CANTIDAD_REGISTROS),
          [x for x in range(1,CANTIDAD_REGISTROS+1)],
          lorem(),
          [1,2,3],
          ['Robo','Asalto','Incendio'],
          [x for x in range(-90,90)],
          [x for x in range(-180,180)]]

RANDOM = [False,
          False,
          True,
          True,
          True,
          True,
          True]

In [10]:
df = pd.DataFrame()
for i in range(len(COLUMNAS)):
    df[COLUMNAS[i]] = [None for x in range(CANTIDAD_REGISTROS)]

for i in range(CANTIDAD_REGISTROS):
    registro = []
    for j in range(len(COLUMNAS)):
        if RANDOM[j]: registro.append(random.choice(FORMATO[j]))
        else: registro.append(FORMATO[j][i])
    df.loc[i] = registro
    
display(df.head(20))
df.to_csv(OUTPUT,index=False,encoding='utf-8')

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2018-01-06,1,Beautiful six already forget today process.,2,Incendio,15,-59
1,2018-01-09,2,Authority American quickly pressure. Step believe,1,Robo,33,172
2,2018-01-10,3,Forward now provide. Up light protect,3,Asalto,-62,-146
3,2018-01-12,4,Town wide near power. Worker young,2,Robo,18,-177
4,2018-01-18,5,Soon various might at. Almost draw,3,Robo,-2,-71
5,2018-01-31,6,Religious lay way eat PM art,3,Robo,32,-157
6,2018-02-15,7,True tell hotel once full investment.,3,Incendio,-45,58
7,2018-02-15,8,Throw high outside that fish. Fact,2,Asalto,14,61
8,2018-02-22,9,Alone check wonder. Everything attention even,3,Robo,-59,93
9,2018-03-16,10,Public name who growth mind. Management,2,Robo,-46,-70


## Formas de importarlo

Pandas

```
import pandas as pd
df = pd.read_csv('data/2018C1_1_GCPD_Casos_Policiales.csv', low_memory=False)
```

Spark

```
import pyspark
sc = pyspask.SparkContext.getOrCreate()
sqlContext = pyspark.sql.SQLContext(sc)
rdd = sqlContext.read.csv('data/2018C1_1_GCPD_Casos_Policiales.csv',header='true').rdd
```

SparkSQL

```
import pyspark
spark = pyspark.sql.SparkSession.builder.appName('Aplicacion').getOrCreate()
df = spark.read.csv('data/2018C1_1_GCPD_Casos_Policiales.csv',header='true')
```